# Community detection

Import the graph as an undirected and unweighted graph.

In [1]:
import networkx as nx
import networkx.algorithms.community as cm
from modules import cytoscape as cs 

G = nx.read_weighted_edgelist('../dataset/gene_edges.tsv', delimiter="\t")

# the "weight" on the graph indicates the
# activation (+1) and deactivation (-1) of  
# genes. We can set all to 1 for community
# detection pourpose 

for u,v,d in G.edges(data=True):
    d['weight']=1

Define a function to save the results

In [15]:
def save_partition(json_partition, name, directory = '../partitions'):
    with open(f'{directory}/{name}.json', 'w') as f:
        f.write(json_partition)

## Clauset-Newman-Moore Modularity maximization

* [Link to the paper](https://arxiv.org/abs/cond-mat/0408187)
* [Link to implementation](https://github.com/networkx/networkx/blob/main/networkx/algorithms/community/modularity_max.py)

In [4]:
cnm_partition = cm.greedy_modularity_communities(G)

In [5]:
cnm_nclusters = len(cnm_partition)
cnm_modularity = cm.modularity(G, cnm_partition)

print(f"cnm produced {cnm_nclusters} clusters with {cnm_modularity:.3f} modularity.")

cnm produced 92 clusters with 0.616 modularity.


In [6]:
cnm_json = cs.partition_to_cytospace_json(G, cnm_partition)
save_partition(cnm_json, 'cnm')

## Girvan-Newman edge-betweenness centrality based 
* [Link to the paper](https://www.pnas.org/content/99/12/7821)
* [Link to implementation](https://github.com/networkx/networkx/blob/main/networkx/algorithms/community/centrality.py)

In [13]:
## Implementare un algoritmo di selezione per la partizione con modularità migliore


# T = nx.karate_club_graph()

gn_communities = cm.girvan_newman(G)

# parameters 
goes_down = 0
goes_down_limit = 5 
iterations = 20

# variables 
partitions_list = []
modularity_list = []

for i in range(iterations): 
    current_partition = next(gn_communities)
    partitions_list.append(current_partition)
    modularity_list.append(cm.modularity(G, current_partition))
    
    if (i > 0 and modularity_list[i-1] > modularity_list[i]):
        goes_down += 1
        if (goes_down > goes_down_limit):
            print(f"stop iteration {i} due to modularity decrease")
            break
    
best_modularity = max(modularity_list)
best_iteration  = modularity_list.index(best_modularity)
best_partition  = partitions_list[best_iteration]

print(best_modularity)

0.04169909755870315


In [14]:
cnm_json = cs.partition_to_cytospace_json(G, best_partition)
save_partition(cnm_json, 'gn') # 0.042 mod 

## FluidC - Asynchronous Fluid Communities algorithm

* [Link to paper](https://arxiv.org/pdf/1703.09307.pdf)
* [Link to implementation](https://github.com/networkx/networkx/blob/main/networkx/algorithms/community/asyn_fluid.py)

Since FluidC works only on connected graphs, we can compute the connected components of $G$ and apply the algorithm on those. 

In [17]:
import numpy as np

ccgen = nx.algorithms.components.connected_components(G) 

# connected components 
cc = [ c for c in sorted( ccgen, key=len, reverse=True ) ]

# calculate the number of clusters to take out from the 
# connected components using the logarithm of its dimension
ncomm = lambda c: int(np.floor(np.log(len(c))))

# initialize an empty set that will be filled with out
# graph partition
partition = []

for ccomponent in cc:
    
    number_of_communities = ncomm(ccomponent)
    
    if (number_of_communities > 1):
        
        # extract the connected component from the graph G
        C = G.subgraph(ccomponent)
        
        # execute the algorithm on the connected subgraph 
        fluidc_community_generator = cm.asyn_fluidc(C, number_of_communities)
        
        # extract the communities
        communities = [ cluster for cluster in fluidc_community_generator ]
        
        # put the communities inside the partition
        partition += communities
        
    else:
        
        # insert the component as a community inside the partition
        partition.append(ccomponent)

In [18]:
cm.modularity(G, partition)
# 0.6339086029546955

0.6339086029546955

In [19]:
fluidc_json = cs.partition_to_cytospace_json(G, partition)
save_partition(fluidc_json, 'fluidc') # 0.042 mod 

## Kernighan-Lin bisection

* [Link to paper](https://ieeexplore.ieee.org/document/6771089)
* [Link to implementation](https://github.com/networkx/networkx/blob/main/networkx/algorithms/community/kernighan_lin.py)

In [53]:
all_nodes = [node for node in G.nodes]

# store the partition of the graph G at
# each level of bisection
dendogram = [[all_nodes]]

# flag to stop the iteration
every_cluster_is_node = False 

# iteration counters 
prev_iter = 0
curr_iter = 1
max_iter  = 20

while (not every_cluster_is_node and curr_iter < max_iter):
    
    curr_partition = []
    prev_partition = dendogram[prev_iter]
    
    # suppose every flag is a single node
    every_cluster_is_node = True
    
    # for each community in the previous partitioning, 
    # apply the bisection if the number of nodes is > 1
    # add the bisection in the current partition 
    for pc in prev_partition:
        subG = G.subgraph(pc)
        if (subG.number_of_nodes() < 2):
            curr_partition.append(pc)
            # if every cluster is a node, in the
            # end this will produce True
            every_cluster_is_node &= True
        else:
            pc_communities = cm.kernighan_lin_bisection(subG)
            curr_partition += pc_communities
            # if there is at least on cluster that
            # is composed of many nodes, then this 
            # flag will be set to False. 
            every_cluster_is_node &= False
    
    dendogram.append(curr_partition)
    prev_iter += 1  
    curr_iter += 1

In [56]:
# remove the first partition (full graph)
valid_partitions = dendogram[1:-1]

# calculate the modularities 
kl_modularities = [ cm.modularity(G, p) for p in dendogram[1:-1] ]

# take the best partition 
best_modl = max(kl_modularities)
best_iter = kl_modularities.index(best_modl)
best_part = valid_partitions[best_iter]

print(best_modl)
#0.061722277284057145

0.061722277284057145


In [57]:
kl_json = cs.partition_to_cytospace_json(G, best_part)
save_partition(kl_json, 'kl') 

## Node2Vec graph embedding

* [Link to paper](https://snap.stanford.edu/node2vec/)
* [Link to implementation](https://github.com/eliorc/node2vec)

In [59]:
from node2vec import Node2Vec

n2v = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4)

In [61]:
model = n2v.fit(window=10, min_count=1, batch_words=4) 

In [63]:
model.wv.save_word2vec_format('n2v/embeddings.emb')
model.save('n2v/embeddings.model')

Let's use kmeans from nodes embedding. 

In [84]:
from sklearn.cluster import KMeans 

# dict: node ID to embedding index 
vocab = model.wv.key_to_index

# nodes embeddings 
vectors = model.wv.vectors 

# Graph nodes IDs 
V = [node for node in G.nodes]

# use kmeans (k=60) on vectors
kmeans = KMeans(n_clusters=60, random_state=0)
kmeans.fit(vectors)

# get the labels (for each node, returns the cluster)
node_labels = kmeans.labels_

# create a partition of G 
n2v_partition = [ [] for _ in range(60) ]
for node in V:
    node_index = vocab.get(node)
    node_cluster = node_labels[node_index]
    n2v_partition[node_cluster].append(node)
    
# print the modularity
cm.modularity(G, n2v_partition)
# 0.6537718239372098

0.6537718239372098

In [85]:
n2v_json = cs.partition_to_cytospace_json(G, n2v_partition)
save_partition(n2v_json, 'n2v') 

## Spectral clustering 
* [Reference (1)](http://citeseer.ist.psu.edu/viewdoc/summary?doi=10.1.1.160.2324)
* [Reference (2)](http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.165.9323)
* [Reference (3)](https://www1.icsi.berkeley.edu/~stellayu/publication/doc/2003kwayICCV.pdf)
* [Link to implementation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html)

In [12]:
from sklearn.cluster import SpectralClustering

def spectral_clustering(G, k, n_init=100):
    A = nx.to_numpy_matrix(G)
    sc = SpectralClustering(
        k, affinity='precomputed', 
        n_init=n_init,
        assign_labels='discretize').fit(A)
    # extract nodes and corresponding labels 
    V, labels = G.nodes(), sc.labels_
    # create the partition and return it 
    partition = [ [] for _ in range(k) ]
    for index, nodeid in enumerate(V):
        label = labels[index]
        partition[label].append(nodeid)
    return partition

In [19]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

partition = spectral_clustering(G, 60, 500)
cm.modularity(G, partition)
# 0.66

0.6535616519292664

In [16]:
sc_json = cs.partition_to_cytospace_json(G, partition)
save_partition(sc_json, 'sc') 

## Louvain - Greedy modularity optimization

* [Link to paper](https://arxiv.org/abs/0803.0476)
* [Link to implementation](https://github.com/taynaud/python-louvain)

In [96]:
import community as louvain

node_to_label = louvain.best_partition(G)

In [100]:
n_cluster = max(node_to_label.values()) + 1
partition = [ [] for i in range(n_cluster) ]

for node, label in partition_as_ids.items():
    partition[label].append(node)
    
cm.modularity(G, partition)
#0.6887668389489657

0.6887668389489657

In [101]:
lvn_json = cs.partition_to_cytospace_json(G, partition)
save_partition(lvn_json, 'lvn') 